In [1]:
import ray
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import xgboost
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from os.path import exists
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import datetime
import numpy as np
import time
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
X = np.loadtxt('data/latent_data_miRNA_L4.csv', delimiter=',')


y = np.loadtxt('data/encoded_labels.csv')

print("Shape of Train Data : {}".format(X.shape))
print("Shape of Test Data : {}".format(y.shape))

input_dim=X.shape[1]
num_classes=len(np.unique(y))

num_epochs=200
batch_size=128
num_P=1 #num_processors per one function

now = datetime.datetime.now()
Time_Now= now.strftime("%Y-%m-%d %H:%M:%S")

dataset= 'PAN'


Shape of Train Data : (6241, 128)
Shape of Test Data : (6241,)


In [3]:
cluster= True
PC=''
if (cluster):
    ray.init(address='auto') # parallel in cluster of PCs
    PC='Cluster'
else:
    ray.init() # parallel in one PC
    PC='Leovo'



2023-03-15 14:56:03,847	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 10.193.5.90:6379...
2023-03-15 14:56:03,894	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


In [4]:
#produce results
path = 'data/results_miRNA_L4/'

def write_results(results, path):
    
    file_exists = exists(path)
    if not(file_exists):
        with open(path,"a+") as f:
            f.write(results)
            f.write("\n")
    else:
        with open(path,"a+") as f:
            f.write(results)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.20)

input_dim=x_train.shape[1]

def create_model():
    
    #early_stop = EarlyStopping(monitor='loss', patience=2)
    model = Sequential()


    model.add(Dense(128, activation='relu', input_shape=(input_dim,),kernel_initializer='random_uniform'))

    model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
    #model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
    #model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    
    return model

2023-03-15 14:56:03.950746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 14:56:04.105370: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-15 14:56:04.105393: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-15 14:56:04.776669: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [6]:
@ray.remote(num_cpus=num_P)
def run_model(i, X,x_train , y_train, x_test , y_test):
    
    t_temp=time.time()
    #individual classifier
    input_dim=x_train.shape[1]

    model1=create_model()
    #hist = model1.fit(x_train , y_train , epochs=20)
    #hist = model1.fit(x_train , y_train , epochs=20, batch_size=5, verbose=0)
    print(x_train.shape)
    hist = model1.fit(x_train , y_train , epochs=num_epochs, batch_size=batch_size, verbose=0)
    y_predict=model1.predict(x_test)
    y_predict=np.argmax(y_predict,axis=1)

    #_, accuracy = model1.evaluate(x_test, y_test)
    #Accuracy
    acc=accuracy_score(y_test, y_predict)
    prc=precision_score(y_test, y_predict, average='weighted')
    rec=recall_score(y_test, y_predict, average='weighted')
    f1=f1_score(y_test, y_predict, average='weighted')
    
 
    results = 'DL:%d' %(i) + ', Acc: %.2f' % (acc*100) + ', Pre: %.2f' % (prc*100)
    results += ', Rec: %.2f' % (rec*100) + ', F1: %.2f' % (f1*100) + ', T:%.2f ' % (time.time()-t_temp) + ', PC: ' + PC
    results += ', Num Processors: ' + str(num_P) + ', Eb:%d ' %(num_epochs) + ', Bs:%d ' %(batch_size) + ', Date:' + Time_Now + '\n'
    
    predictions=model1.predict(X)
    max_indices = np.argmax(predictions,axis=1)

    return_values=[]
    return_values.append(np.array(max_indices))
    return_values.append(results)

    return return_values

t1=time.time()

Return_values=[]

Return_values=[run_model.remote(i, X,x_train , y_train, x_test , y_test) for i in range(5)]

#collect the output from classifiers that used original features

outputs_original=ray.get(Return_values)

all_time_before=time.time()-t1

#0 indicates to the the outputs_original
#1 indicates to results_original
predictions=outputs_original[0][0]
for i in range(1,5):
    predictions=np.vstack((predictions,outputs_original[i][0]))

predictions= np.transpose(predictions)

# concat predictions with olriginal features
new_X=np.concatenate((X, predictions), axis=1)
#new_X=predictions #np.concatenate((X, predictions), axis=1)
#====z_score normalization
scaler = StandardScaler()
scaler.fit(new_X)
new_X = scaler.fit_transform(new_X)
print(new_X.shape, predictions.shape)

new_x_train,new_x_test,y_train,y_test = train_test_split(new_X,y,test_size = 0.10)

input_dim=new_x_train.shape[1]


#Save the performance results of original classifiers
path2= path +"Res_Singl_Parallel_DL_Original.txt" # the name of the file
results_before='Dataset:' + dataset + '=============================================\n'

for i in range(5):
    results_before+=outputs_original[i][1]

results_before+= 'Done...in time:' + str(round(all_time_before)) + '\n'
    
write_results(results_before, path2)


(pid=396770) 2023-03-15 14:56:06.483684: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=396770) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=396788) 2023-03-15 14:56:06.494704: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=396788) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=396771) 2023-03-15 14:56:06.572854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(

(run_model pid=396788) (4992, 128)
(run_model pid=396770) (4992, 128)
(run_model pid=396771) (4992, 128)


(run_model pid=396807) 2023-03-15 14:56:10.447164: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(run_model pid=396807) 2023-03-15 14:56:10.447219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alandoli-Lenovo-Z50-70): /proc/driver/nvidia/version does not exist
(run_model pid=396807) 2023-03-15 14:56:10.447674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(run_model pid=396807) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(run_model pid=396807) (4992, 128)


(pid=9472, ip=10.193.15.118) 2023-03-15 14:56:17.409297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=9472, ip=10.193.15.118) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=9472, ip=10.193.15.118) 2023-03-15 14:56:17.606723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=9472, ip=10.193.15.118) 2023-03-15 14:56:17.606759: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(pid=9472, ip=10.193.15.118) 2023-03-15 14:56:18.325034: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

...

40/40 [==============================] - 0s 3ms/step


(run_model pid=396807) /home/alandoli/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
(run_model pid=396807)   _warn_prf(average, modifier, msg_start, len(result))


40/40 [==============================] - 0s 1ms/step


(run_model pid=396770) /home/alandoli/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
(run_model pid=396770)   _warn_prf(average, modifier, msg_start, len(result))
(run_model pid=396770) /home/alandoli/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
(run_model pid=396770)   _warn_prf(average, modifier, msg_start, len(result))


196/196 [==============================] - 0s 889us/step
(6241, 133) (6241, 5)


In [7]:
input_dim=new_x_train.shape[1]

NN_clf1=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf1._estimator_type = "classifier"
NN_clf2=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf2._estimator_type = "classifier"
NN_clf3=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf3._estimator_type = "classifier"
NN_clf4=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf4._estimator_type = "classifier"
NN_clf5=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf5._estimator_type = "classifier"


/tmp/ipykernel_396687/3367525961.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  NN_clf1=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
/tmp/ipykernel_396687/3367525961.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  NN_clf2=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
/tmp/ipykernel_396687/3367525961.py:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  NN_clf3=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0

In [8]:
dtc =  DecisionTreeClassifier()
rfc = RandomForestClassifier()
knn =  KNeighborsClassifier()
xgb = xgboost.XGBClassifier()
svm = SVC(probability=True)

#intermediate = [('NN1', NN_clf1), ('NN2', NN_clf2), ('NN3', NN_clf3), ('NN4', NN_clf4), ('NN5', NN_clf5),('dtc',dtc),('rfc',rfc),('knn',knn) ,('xgb',xgb),('svm',svm)]
intermediate = [('NN1', NN_clf1), ('NN2', NN_clf2), ('NN3', NN_clf3),
                ('NN4', NN_clf4), ('NN5', NN_clf5),('dtc',dtc),('rfc',rfc),
                ('knn',knn) ,('xgb',xgb)]
intermediate


[('NN1', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa893f853c0>),
 ('NN2', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa893f84b20>),
 ('NN3', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa893f84ac0>),
 ('NN4', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa9f7ec1f00>),
 ('NN5', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa893f84f70>),
 ('dtc', DecisionTreeClassifier()),
 ('rfc', RandomForestClassifier()),
 ('knn', KNeighborsClassifier()),
 ('xgb',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
            

In [9]:
# run after ad preictions of first level
t1=time.time()
@ray.remote(num_cpus=num_P)
def parallel_ensemble(algo,clf_name, p_X,p_y):
    t_temp=time.time()
    y_predict = cross_val_predict(algo,p_X,p_y, cv=5) 
    #score = cross_val_score( algo,new_X,y,cv = 10,scoring = 'accuracy')
    #print("The accuracy score of {} is:".format(algo),score.mean(), "time: ", time.time()-t_temp)
    acc=accuracy_score(y_predict,y)
    prc=precision_score(y_predict,y, average='weighted')
    rec=recall_score(y_predict,y, average='weighted')
    f1=f1_score(y_predict,y, average='weighted')


    results = 'Model: ' + clf_name + ', Acc: %.2f' % (acc*100) + ', Pre: %.2f' % (prc*100)
    results += ', Rec: %.2f' % (rec*100) + ', F1: %.2f' % (f1*100) + ', T:%.2f ' % (time.time()-t_temp) + ', PC: ' + PC
    results += ', Num Processors: ' + str(num_P) + ', Eb:%d ' %(num_epochs) + ', Bs:%d ' %(batch_size) + ', Date:' + Time_Now + '\n'
    
    return results

clf = [NN_clf1,NN_clf2,NN_clf3,NN_clf4,NN_clf5, dtc, rfc, knn, xgb, svm]
clf_name = ['NN_clf1','NN_clf2','NN_clf3','NN_clf4', 'NN_clf5', 'dtc', 'rfc', 'knn', 'xgb', 'svm']

outputs_after=[parallel_ensemble.remote(clf[i], clf_name[i], new_X,y) for i in range (len(clf_name))]

    
#collect the results of classification after get new features from the bae classifier.
path2=path + "Res_Singl_Parallel_ML_DL_after.txt" # the name of the file
results_after='Dataset:' + dataset + '=============================================\n'

for i in range(len(clf_name)):
    results_after+=ray.get(outputs_after[i])
    
all_time_after=time.time()-t1

results_after+='Done...in time:' + str(round(all_time_after)) + '\n'
write_results(results_after, path2)


39/39 [==============================] - 0s 2ms/step18)[0


(parallel_ensemble pid=9555, ip=10.193.15.118) /home/alandoli/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
(parallel_ensemble pid=9555, ip=10.193.15.118)   _warn_prf(average, modifier, msg_start, len(result))


39/39 [==============================] - 0s 1ms/step18) 


(parallel_ensemble pid=9648, ip=10.193.15.118) /home/alandoli/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
(parallel_ensemble pid=9648, ip=10.193.15.118)   _warn_prf(average, modifier, msg_start, len(result))


39/39 [==============================] - 0s 931us/step


In [10]:
#Ensemble stacked results
t2=time.time()
#kfold = KFold(n_splits=10, shuffle=True)
clf = StackingClassifier(estimators=intermediate, final_estimator=LogisticRegression()) #MLPClassifier()

clf.fit(new_x_train , y_train)
#score = cross_val_score(stack_model,new_X,y,cv = 10,scoring = 'accuracy')
#score
y_predict=clf.predict(new_x_test)
acc=accuracy_score(y_predict,y_test)
prc=precision_score(y_predict,y_test, average='weighted')
rec=recall_score(y_predict,y_test, average='weighted')
f1=f1_score(y_predict,y_test, average='weighted')

Ense_results ='Acc: %.2f' % (acc*100) + ', Pre: %.2f' % (prc*100)
Ense_results += ', Rec: %.2f' % (rec*100) + ', F1: %.2f' % (f1*100) + ', PC: ' + PC
Ense_results += ', Num Processors: ' + str(num_P) +  ', Eb:%d ' %(num_epochs) + ', Bs:%d ' %(batch_size) + 'Dataset: ' + dataset  + ', Date:' + Time_Now + '\n'
  
Ense_results+='Done...in time:' + str(round(time.time()-t2)) + '\n'

path2= path + "Res_Parallel_Ensemble_ML_DL.txt" # the name of the file
write_results(Ense_results, path2)



print("Done...in time: ", time.time()-t2)

2023-03-15 15:05:05.122193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-15 15:05:05.122247: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alandoli-Lenovo-Z50-70): /proc/driver/nvidia/version does not exist
2023-03-15 15:05:05.122685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


20/20 [==============================] - 0s 1ms/step
Done...in time:  748.5177056789398


/home/alandoli/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
print("Stacking model score", clf.score(new_x_test , y_test))


20/20 [==============================] - 0s 1ms/step
Stacking model score 0.888


In [12]:
ray.shutdown()